## Introduction

I am using my home PC, with Debian 12 "bookworm" (testing) and an NVIDIA GPU.

The following setup is **not recommended** unless you know what you're doing with Linux, and probably not even then.

I decided not to use conda, virtualenv, or docker. Instead, I installed the necessary Python modules under `/usr/local` with `pip`. This way I can use my AI tools directly from the command line and other scripts, like any other tools.

## Allow myself to install packages to /usr/local


It's not safe to run `pip install -U` as root. It will merrily remove files from Debian-packaged Python modules under `/usr/lib`, and mess up the system. To avoid this, I changed permissions so that my regular user account can write to everything under `/usr/local` via the `staff` group. I do the same for `/opt` because I want to upgrade Rust from my own user ID also:

In [ ]:
sudo adduser $USER staff
sudo chgrp -R staff /usr/local /opt
sudo chmod -R g+w /usr/local /opt

This is somewhat of a security concern, but if a bad guy gets a local shell it's pretty much game over anyway. To make it safer, I could have used a different non-root account.

In order to upgrade a pip package where the same package was also installed in `/usr/lib` by dpkg, I need to use `pip -I -U packagename`. The `-I` flag tells it to ignore installed packages, i.e. don't try to remove them. I only do that as needed for individual packages, when a normal upgrade fails.

This setup might upset Debian-packaged programs that depend on older versions of the Python libraries that I've upgraded. I haven't noticed any problems like that yet.

If you try to downgrade a package, and If multiple versions of a package are accidentally installed under /usr/local/lib/python3.10 at that same time, the sym

## Enable switching the system python version

The default Python version for Debian "testing" at time of writing is Python 3.11, however we don't have a stable release of pytorch for Python 3.11 yet:

In [ ]:
python3.11 -m pip install torchvision

ERROR: Could not find a version that satisfies the requirement torchvision (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3)
ERROR: No matching distribution found for torchvision



: 1

I'm leaving this section in, however I have since switched to using Python 3.11 and nightly builds of torch.

Anyway, let's enable switching the system Python version back to 3.10, using `update-alternatives`.

First, note that we can undo this change later, if necessary, as follows:

In [ ]:
sudo update-alternatives --remove-all python3
sudo update-alternatives --remove-all pydoc3
sudo ln -sf python3.11 /usr/bin/python3
sudo ln -sf pydoc3.11 /usr/bin/pydoc3

We can set up alternatives for `python3`:

In [ ]:
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 11
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 10
# sudo update-alternatives --set python3 /usr/bin/python3.10

update-alternatives: using /usr/bin/python3.11 to provide /usr/bin/python3 (python3) in auto mode
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in manual mode


and do the same for `pydoc3`:

In [ ]:
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.11 11
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.10 10
# sudo update-alternatives --set pydoc3 /usr/bin/pydoc3.10

update-alternatives: using /usr/bin/pydoc3.11 to provide /usr/bin/pydoc3 (pydoc3) in auto mode
update-alternatives: using /usr/bin/pydoc3.10 to provide /usr/bin/pydoc3 (pydoc3) in manual mode


In [ ]:
update-alternatives --list python3

/usr/bin/python3.10
/usr/bin/python3.11


In [ ]:
update-alternatives --list pydoc3

/usr/bin/pydoc3.10
/usr/bin/pydoc3.11


## Install Rust

Some Python modules now depend on Rust to build. I want to install Rust system-wide, in `/opt/rust`:

In [ ]:
cd /tmp
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs > install_rust.sh
sudo RUSTUP_HOME=/opt/rust CARGO_HOME=/opt/rust sh ./install_rust.sh -y --no-modify-path

info: downloading installer
info: (Bprofile set to 'default'
info: (Bdefault host triple is x86_64-unknown-linux-gnu
info: (Bsyncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: (Bdefault toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu unchanged(B - rustc 1.67.1 (d5a82bbd2 2023-02-07)


Rust is installed now. Great!
(B
To get started you need Cargo's bin directory (/opt/rust/bin) in your PATH(B
environment variable. This has not been done automatically.

To configure your current shell, run:
source "/opt/rust/env"


In [ ]:
. /opt/rust/env
rustup default stable

info: (Busing existing install for 'stable-x86_64-unknown-linux-gnu'
info: (Bdefault toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu unchanged(B - rustc 1.67.1 (d5a82bbd2 2023-02-07)



## Install packages for torch, fastai and jupyter

In [ ]:
pip install -qq -U fastbook jupyter jupyterlab tensorflow ipywidgets

At some point I saw a warning about compatibility between fastai and torch.
```
fastai 2.7.11 has requirement torch<1.14,>=1.7, but you have torch 2.0.0.dev20230220+cu117
```
I decided to try using it with nightly torch anyway:

In [ ]:
pip install -qq -U --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu117

Extra packages: nbdev, and kernels for bash and C:

In [ ]:
pip install -qq -U nbdev bash_kernel jupyter-c-kernel

In [ ]:
python -m bash_kernel.install

Installing IPython kernel spec


In [ ]:
install_c_kernel

Installing IPython kernel spec
/usr/local/bin/install_c_kernel:32: DeprecationWarning: replace is ignored. Installing a kernelspec always replaces an existing installation
  KernelSpecManager().install_kernel_spec(td, 'c', user=user, replace=True, prefix=prefix)


## Untold difficulties

I haven't described all of the various problems I've had to deal with, due to using this approach. There were plenty of problems! I guess I'm learning something from it, anyway. I wish Lambda Stack worked on Debian, not only Ubuntu, it would be a lot easier than this. Maybe I should give it a try on Debian anyway, and see what happens.

## Use the old pip resolver, if necessary

As of pip 20.3, a new resolver has been introduced, which doesn't always work. As of pip 21.0 the old working resolver is unsupported and slated for removal dependent on pip team resources.

In [ ]:
pip --version

pip 23.0 from /usr/lib/python3/dist-packages/pip (python 3.11)


With this version of pip, it's possible to use the "legacy resolver" with the following option:

In [ ]:
pip install -U -r requirements.txt --use-deprecated=legacy-resolver

Or, I can downgrade pip to version 20.2.4, before they switched to the new resolver.

Refer to https://stackoverflow.com/a/67408694 for more info.

In [ ]:
pip install pip==20.2.4

  Using cached pip-20.2.4-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'pip'. No files were found to uninstall.


In [ ]:
pip --version

pip 20.2.4 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


## NVIDIA tensorrt is not ready for python3.11

In [ ]:
sudo apt-get -q install python3-libnvinfer-dev

Reading package lists...
Building dependency tree...
Reading state information...
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 python3-libnvinfer : Depends: python3 (< 3.11) but 3.11.1-3 is to be installed
E: Unable to correct problems, you have held broken packages.


: 100

I'd like to install it as a python3.10 library at least, but it is demanding that the system Python version, i.e. the version of Debian's python3 package, should be less than 3.11, and I don't want to try to change that. So I'll adjust this `python3-libnvinfer` package to depend on the `python3.10` package instead of `python3 < 3.11`.

In [ ]:
cd /var/nv-tensorrt-local-repo-ubuntu2204-8.5.3-cuda-11.8
ls python3-libnvinfer*

python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb
python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb


In [ ]:
cp -v python3-libnvinfer* ~/soft-ai/

'python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb' -> '/home/sam/soft-ai/python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb'
'python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb' -> '/home/sam/soft-ai/python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb'


In [ ]:
cd ~/soft-ai
dpkg-deb -R python3-libnvinfer_8.5.3-1+cuda11.8_amd64.deb unpacked

In [ ]:
grep Depends unpacked/DEBIAN/control

Depends: python3 (>= 3.10), python3 (<< 3.11), libnvinfer8 (= 8.5.3-1+cuda11.8), libnvinfer-plugin8 (= 8.5.3-1+cuda11.8), libnvparsers8 (= 8.5.3-1+cuda11.8), libnvonnxparsers8 (= 8.5.3-1+cuda11.8)


In [ ]:
sed -i 's/python3 (>= 3.10), python3 (<< 3.11), /python3.10, /' unpacked/DEBIAN/control
grep Depends unpacked/DEBIAN/control

Depends: python3.10, libnvinfer8 (= 8.5.3-1+cuda11.8), libnvinfer-plugin8 (= 8.5.3-1+cuda11.8), libnvparsers8 (= 8.5.3-1+cuda11.8), libnvonnxparsers8 (= 8.5.3-1+cuda11.8)


In [ ]:
dpkg-deb -b unpacked python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb

dpkg-deb: building package 'python3-libnvinfer' in 'python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb'.


In [ ]:
sudo dpkg -i ./python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb ./python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb

Selecting previously unselected package python3-libnvinfer.
(Reading database ... 1135689 files and directories currently installed.)
Preparing to unpack .../python3-libnvinfer_8.5.3-1+cuda11.8_amd64_fixed.deb ...
Unpacking python3-libnvinfer (8.5.3-1+cuda11.8) ...
Selecting previously unselected package python3-libnvinfer-dev.
Preparing to unpack .../python3-libnvinfer-dev_8.5.3-1+cuda11.8_amd64.deb ...
Unpacking python3-libnvinfer-dev (8.5.3-1+cuda11.8) ...
Setting up python3-libnvinfer (8.5.3-1+cuda11.8) ...
Setting up python3-libnvinfer-dev (8.5.3-1+cuda11.8) ...


## PyPI does not include onnxruntime for python3.11

## Python environments

I'm going to try setting up a series of Python environments under `/opt`, and adding them to the PYTHONPATH as appropriate.

It will be easier to rebuild them rather than 

| Folder | Depends | Uses | Conflicts |
| --- | --- | --- | --- |
| /opt/python3.11-ai | Debian's default python3.11 | torch 2 (nightly) | fastai, tensorrt |
| /opt/python3.10-ai | Debian's python3.10 | torch stable () |  |
| /opt/python3.10-fastai | python3.10-fastai | torch stable () |  |
| /opt/python3.10-webui | python3.10-ai | torch stable () |  |

In [ ]:
cw python_env

#!/bin/bash
echo $VIRTUAL_ENV
python -V
which python
namei `which python` | sed -n '/->/ { s/.*-> //; p; }'
python -c 'import sys; print(sys.path)'


In [ ]:
python_env


Python 3.11.2
/usr/local/bin/python
/usr/bin/python
python3
/etc/alternatives/python3
/usr/bin/python3.11
['', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '/usr/lib/python3/dist-packages', '/usr/lib/python3.11/dist-packages']


In [ ]:
cd /opt
ls -ld python*

drwxr-xr-x 5 sam sam 4096 Feb 28 05:50 python3.10-ai


In [ ]:
command rm -rf ./python3.10-ai

In [ ]:
python3.10 -m venv python3.10-ai

In [ ]:
. /opt/python3.10-ai/bin/activate

(python3.10-ai) 


: 1

## PyPI does not include onnxruntime for python3.11

In [ ]:
deactivate

## Maybe use /usr/local installs first then /opt venv installs based on those. no

- the /usr/local/lib/python3.10 install doesn't work, though
- /usr/local/lib/python3.11/dist-packages
- /usr/local/lib/python3.10/dist-packages